# Single-cell RNA-seq imputation using DeepImpute

Here is a comprehensive tutorial to understand the functionnalities of DeepImpute.

In [1]:
from deepimpute.multinet import MultiNet
import pandas as pd

# Load dataset using pandas
data = pd.read_csv('test.csv',index_col=0)
print('Working on {} cells and {} genes'.format(*data.shape))

Using TensorFlow backend.


Working on 500 cells and 3000 genes


## Create a DeepImpute multinet

In [2]:
# Using default parameters
multinet = MultiNet() 

Using all the cores (4)


In [4]:
# Using custom parameters
NN_params = {
        'learning_rate': 1e-4,
        'batch_size': 64,
        'max_epochs': 200,
        'ncores': 5,
        'sub_outputdim': 512,
        'architecture': [
            {"type": "dense", "activation": "relu", "neurons": 200},
            {"type": "dropout", "activation": "dropout", "rate": 0.3}]
    }

multinet = MultiNet(**NN_params)

## Fit the networks

In [ ]:
# Using all the data
multinet.fit(data,cell_subset=1,minVMR=0.5)

In [5]:
# Using 80% of the data
multinet.fit(data,cell_subset=0.5)

3072 genes selected for imputation
Net 0: 848 predictors, 512 targets
Net 1: 839 predictors, 512 targets
Net 2: 841 predictors, 512 targets
Net 3: 841 predictors, 512 targets
Net 4: 824 predictors, 512 targets
Net 5: 763 predictors, 512 targets
Normalization
Building network
[{'type': 'dense', 'activation': 'relu', 'neurons': 200}, {'type': 'dropout', 'activation': 'dropout', 'rate': 0.3}]
Fitting with 250 cells
Train on 238 samples, validate on 12 samples
Epoch 1/200
238/238 [==============================] - 1s 5ms/step - loss: 27.9152 - dense_19_loss: 5.2994 - dense_20_loss: 4.3443 - dense_21_loss: 4.4147 - dense_22_loss: 4.5893 - dense_23_loss: 3.8626 - dense_24_loss: 5.4048 - val_loss: 32.0546 - val_dense_19_loss: 6.0331 - val_dense_20_loss: 5.0119 - val_dense_21_loss: 5.1475 - val_dense_22_loss: 5.1163 - val_dense_23_loss: 4.5323 - val_dense_24_loss: 6.2136
Epoch 2/200
238/238 [==============================] - 0s 433us/step - loss: 25.1458 - dense_19_loss: 4.8155 - dense_20_loss

Epoch 20/200
238/238 [==============================] - 0s 393us/step - loss: 8.8702 - dense_19_loss: 1.7462 - dense_20_loss: 1.3634 - dense_21_loss: 1.3565 - dense_22_loss: 1.4306 - dense_23_loss: 1.2222 - dense_24_loss: 1.7512 - val_loss: 6.2637 - val_dense_19_loss: 1.1666 - val_dense_20_loss: 0.9941 - val_dense_21_loss: 1.0558 - val_dense_22_loss: 0.9726 - val_dense_23_loss: 0.9219 - val_dense_24_loss: 1.1527
Epoch 21/200
238/238 [==============================] - 0s 509us/step - loss: 8.4533 - dense_19_loss: 1.6640 - dense_20_loss: 1.3292 - dense_21_loss: 1.2929 - dense_22_loss: 1.3470 - dense_23_loss: 1.1747 - dense_24_loss: 1.6455 - val_loss: 5.9226 - val_dense_19_loss: 1.0918 - val_dense_20_loss: 0.9434 - val_dense_21_loss: 1.0007 - val_dense_22_loss: 0.9270 - val_dense_23_loss: 0.8629 - val_dense_24_loss: 1.0969
Epoch 22/200
238/238 [==============================] - 0s 388us/step - loss: 8.2901 - dense_19_loss: 1.6359 - dense_20_loss: 1.2951 - dense_21_loss: 1.2787 - dense_22_

Epoch 40/200
238/238 [==============================] - 0s 526us/step - loss: 5.5575 - dense_19_loss: 1.0798 - dense_20_loss: 0.9070 - dense_21_loss: 0.8272 - dense_22_loss: 0.9102 - dense_23_loss: 0.7706 - dense_24_loss: 1.0627 - val_loss: 2.8307 - val_dense_19_loss: 0.4794 - val_dense_20_loss: 0.4736 - val_dense_21_loss: 0.4542 - val_dense_22_loss: 0.4747 - val_dense_23_loss: 0.4348 - val_dense_24_loss: 0.5140
Epoch 41/200
238/238 [==============================] - 0s 419us/step - loss: 5.4591 - dense_19_loss: 1.0370 - dense_20_loss: 0.8730 - dense_21_loss: 0.8274 - dense_22_loss: 0.8988 - dense_23_loss: 0.7622 - dense_24_loss: 1.0607 - val_loss: 2.7543 - val_dense_19_loss: 0.4724 - val_dense_20_loss: 0.4807 - val_dense_21_loss: 0.4536 - val_dense_22_loss: 0.4488 - val_dense_23_loss: 0.4189 - val_dense_24_loss: 0.4800
Epoch 42/200
238/238 [==============================] - 0s 471us/step - loss: 5.4187 - dense_19_loss: 1.0430 - dense_20_loss: 0.8719 - dense_21_loss: 0.8212 - dense_22_

Epoch 60/200
238/238 [==============================] - 0s 417us/step - loss: 4.3803 - dense_19_loss: 0.8178 - dense_20_loss: 0.7100 - dense_21_loss: 0.6657 - dense_22_loss: 0.7328 - dense_23_loss: 0.6123 - dense_24_loss: 0.8416 - val_loss: 2.1896 - val_dense_19_loss: 0.3797 - val_dense_20_loss: 0.3904 - val_dense_21_loss: 0.3412 - val_dense_22_loss: 0.3592 - val_dense_23_loss: 0.3548 - val_dense_24_loss: 0.3642
Epoch 61/200
238/238 [==============================] - 0s 392us/step - loss: 4.3089 - dense_19_loss: 0.8247 - dense_20_loss: 0.6944 - dense_21_loss: 0.6420 - dense_22_loss: 0.7347 - dense_23_loss: 0.5995 - dense_24_loss: 0.8137 - val_loss: 2.2014 - val_dense_19_loss: 0.3569 - val_dense_20_loss: 0.3916 - val_dense_21_loss: 0.3773 - val_dense_22_loss: 0.3536 - val_dense_23_loss: 0.3497 - val_dense_24_loss: 0.3724
Epoch 62/200
238/238 [==============================] - 0s 368us/step - loss: 4.2898 - dense_19_loss: 0.8172 - dense_20_loss: 0.7040 - dense_21_loss: 0.6546 - dense_22_

Epoch 80/200
238/238 [==============================] - 0s 380us/step - loss: 3.6924 - dense_19_loss: 0.7004 - dense_20_loss: 0.5997 - dense_21_loss: 0.5605 - dense_22_loss: 0.6195 - dense_23_loss: 0.5309 - dense_24_loss: 0.6811 - val_loss: 1.9189 - val_dense_19_loss: 0.3111 - val_dense_20_loss: 0.3415 - val_dense_21_loss: 0.3063 - val_dense_22_loss: 0.3252 - val_dense_23_loss: 0.2914 - val_dense_24_loss: 0.3434
Epoch 81/200
238/238 [==============================] - 0s 444us/step - loss: 3.6229 - dense_19_loss: 0.6849 - dense_20_loss: 0.5962 - dense_21_loss: 0.5546 - dense_22_loss: 0.5963 - dense_23_loss: 0.5171 - dense_24_loss: 0.6738 - val_loss: 2.0380 - val_dense_19_loss: 0.3340 - val_dense_20_loss: 0.3518 - val_dense_21_loss: 0.3202 - val_dense_22_loss: 0.3459 - val_dense_23_loss: 0.3497 - val_dense_24_loss: 0.3365
Epoch 82/200
238/238 [==============================] - 0s 382us/step - loss: 3.6819 - dense_19_loss: 0.6907 - dense_20_loss: 0.6065 - dense_21_loss: 0.5579 - dense_22_

In [ ]:
# Using 200 cells (randomly selected)
multinet.fit(data,cell_subset=200)

In [ ]:
# Custom fit
trainingData = data.iloc[100:250,:]
multinet.fit(trainingData)

## Imputation

The imputation can be done on any dataset as long as the gene labels are the same

In [ ]:
imputedData = multinet.predict(data)

## Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

limits = [0,100]

fig,ax = plt.subplots()

jitter = np.random.normal(0,1,data.size) # Add some jittering to better see the point density
ax.scatter(data.values.flatten()+jitter,imputedData.values.flatten(),s=2)
ax.plot(limits,limits,'r-.',linewidth=2)
ax.set_xlim(limits)
ax.set_ylim(limits)

plt.show()


## Scoring
Display training metrics (MSE and Pearson's correlation on the test data)

In [ ]:
multinet.test_metrics